# Editionsdaten semantisch anreichern – XML-Parsing und Named Entity Recognition (NER) mit Python

Dieses Notebook ist im Rahmen der Fortbildungsreihe "Vom Dokument zur Edition" und konkret des Workshops [Editionsdaten semantisch anreichern – XML-Parsing und Named Entity Recognition (NER) mit Python](https://www.it.fu-berlin.de/unsere-services/kompetenzentwicklung/fortbildungen/workshops/E-Research/2026-01-27-Editionsdaten-WS4.html) im WiSe 2025/26 an der FU Berlin entstanden. Das Notebook soll als erster Einstieg dienen und erklären, wie XML-Dateien mit Python eingelesen sowie der enthaltene Text extrahiert, weiterverarbeitet und ins ursprüngliche XML zurückgeführt werden kann.

Beitragende anhand der [Contributor Role Taxonomy (CRediT) taxonomy](https://doi.org/10.1002/leap.1210):
* Sophie Schneider ([@BibWiss](https://github.com/BibWiss)): Conceptualization, Software/Writing - Original Draft (Teil 1 & 2), Writing - Review & Editing (Teil 3)
* Catherine Anne Seveke ([@cati-gitling](https://github.com/cati-gitling)): Conceptualization, Software/Writing - Original Draft (Teil 3)

## Teil 1: XML einlesen und aufbereiten

### 1. Installation
Um dieses Notebook ausführen zu können, müssen [Python](https://www.python.org/) und [Jupyter Notebook](https://jupyter.org/) installiert sein. In Vorbereitung auf die Veranstaltung wird den Teilnehmenden eine darauf zugeschittene Anleitung zur Installation von Anaconda zugeschickt. Mit Anaconda sind neben Python bereits Jupyter Notebook sowie diverse weitere Bibliotheken vorinstalliert. Da die Bibliothek spacy (welche wir benötigen) leider nicht in der Liste der vorinstallierten Bibliotheken enthalten ist, erstellen wir eine seperate Entwicklungsumgebung. 

Hierzu öffnen wir ein Kommandozeilenfenster (*CMD.exe Prompt*) und geben die folgenden Befehle jeweils nacheinander ein:

* `conda create -n ner4xml python=3.11`
* `y`
* `conda activate ner4xml`
* `conda install jupyter spacy pandas nltk`
* `jupyter notebook`

Kurzzusammenfassung, was hier passiert ist: wir (1) erstellen mit dem Paketmanager `conda` eine neue Umgebung namens `ner4xml`, welche die Python-Version 3.11 nutzt, mit (2) `y` (`yes`) bestätigen wir den Vorgang und sobald dies beendet ist (3) aktivieren wir die Umgebung, (4) installieren alle benötigten Bibliotheken und (5) rufen Jupyter Notebook auf. Dann müssen wir noch zu unserem Notebook navigieren und können dieses öffnen. Weitere Befehle, die sonst im Terminal eingegeben werden müssten (z.B. zur Installation weiterer Bibliotheken), können wir nun auch im Notebook aufrufen, indem wir ein `!` voranstellen.

In [1]:
#ein typischer Befehl, um zu prüfen ob und welche Version von Python installiert ist
!python --version

Python 3.9.23


Nun laden wir für später noch ein Modell von spacy herunter (diesen Schritt müssen wir nur einmal ausführen, dann können wir ihn z.B. mithilfe von `#` auskommentieren):

In [2]:
#!python -m spacy download en_core_web_sm

### 2. Import der Bibliotheken
Während die Bibliotheken grundsätzlich nur einmal installiert werden müssen, müssen wir sie jedes Mal vor der Verwendung importieren. Haben wir den Import vergessen, kommt z.B. eine Fehlermeldung wie `NameError: name '<name_bibliothek>' is not defined` 😡. Um das zu vermeiden, müssen wir alle Python-Bibliotheken importieren - auch solche aus der Standardbibliothek (wie hier `xml.etree.ElementTree`), die wir gar nicht gesondert vorher installieren mussten. Das Importieren erledigen wir am besten immer direkt am Anfang eines Python-Skripts oder -Notebooks ✅

In [3]:
import xml.etree.ElementTree as ET
import pandas as pd
import spacy

### 3. Laden und Parsen der XML-Daten
Nun geht es ans Eingemachte 🫙. Wir definieren eine Variable `path_to_file` und geben dort den relativen Pfad zur XML-Datei (kann abweichen und muss ggf. angepasst werden!) an. Mit `ET.parse()` wird die Datei geparsed (eingelesen) und mit `getroot()` sprechen wir den Wurzelknoten des XMLs an.

In [4]:
path_to_file = "data/input.xml"

tree = ET.parse(path_to_file)
root = tree.getroot()

Mit der `print()`-Funktion können wir eine Ausgabe erzeugen und uns hier beispielsweise das Element an der Wurzel anzeigen lassen. Wir sehen, dass vor dem Elementnamen noch eine Information über den Namespace enthalten ist: `{http://www.tei-c.org/ns/1.0}`. Das verwundert vielleicht zunächst, da es im XML-Dokument nicht an dieser Position zu lesen war. Damit ET 👽 weiß, dass wir uns auf diesen Namensraum beziehen, ohne dass wir nun jedes Mal diese Information voranstellen müssen, um einen Knoten ansprechen zu können, definieren wir eine Variable `namespaces` auf die wir später noch zugreifen werden. 

In [5]:
print(root)
print(root.tag)

namespaces={"":"http://www.tei-c.org/ns/1.0"}

<Element '{http://www.tei-c.org/ns/1.0}TEI' at 0x7f8a382fd6d0>
{http://www.tei-c.org/ns/1.0}TEI


Nun sammeln wir alle Informationen, die wir gleich noch benötigen könnten, und legen dafür drei Listen an: `xml_nodes` für die Knoten der Elemente, auf die wir uns später beziehen und die wir ggf. auch aktualisieren wollen (alle, die in irgendeiner Weise Text enthalten), `xml_tags` für die Tag-Namen der gleichen Knoten und `xml_text` für den darin enthaltenen Text.

In [6]:
xml_nodes = []
xml_tags = []
xml_text = []

Jetzt starten wir unseren ersten Loop 💫 - glücklichweise bietet ET dafür auch direkt die Funktion `iter()` an. Wir machen zunächst den ersten (einzigen) Textknoten ausfindig (siehe auch: https://docs.python.org/3/library/xml.etree.elementtree.html#supported-xpath-syntax), iterieren durch dessen (Kind-)Elemente und speichern den mit `strip()` am Anfang und Ende von Leerzeichen befreiten Text dieser Elemente jeweils als `descendant_text`. Dann checken wir nochmal mit einer `if`-Bedingung ob auch wirklich Text enthalten ist und wenn dem so ist, fügen wir jeder der Listen je einen entsprechenden Eintrag hinzu. Am Ende checken wir nochmal ein paar Einträge und lassen uns die Gesamtlänge der ersten Liste (die anderen beiden müssten genau gleich lang sein) ausgeben.

In [7]:
#find nodes
title_node = root.findall(".//title", namespaces)[0]
text_node = root.findall(".//text", namespaces)[0]

In [8]:
#add nodes / tags / texts to list
xml_nodes.append(title_node)
xml_tags.append(title_node.tag)
xml_text.append(title_node.text)

In [9]:
#add nodes and information iteratively
for descendant in text_node.iter(): #for the full document, including metadata, use root.iter()
      descendant_text = str(descendant.text).strip()
      if descendant_text != "" and descendant_text != "None":
            xml_nodes.append(descendant)
            xml_tags.append(descendant.tag)
            xml_text.append(descendant.text)

print([(node, tag, text) for (node, tag, text) in zip(xml_nodes[:3], xml_tags[:3], xml_text[:3])])
print(len(xml_nodes))

[(<Element '{http://www.tei-c.org/ns/1.0}title' at 0x7f8a382fd950>, '{http://www.tei-c.org/ns/1.0}title', 'H. P. Lovecraft: Astronomical Observations (NAME)'), (<Element '{http://www.tei-c.org/ns/1.0}expan' at 0x7f8a382309f0>, '{http://www.tei-c.org/ns/1.0}expan', 'Observed'), (<Element '{http://www.tei-c.org/ns/1.0}abbr' at 0x7f8a38230a90>, '{http://www.tei-c.org/ns/1.0}abbr', 'Obs’d')]
406


### 4. Daten mit pandas aufbereiten

Der Vorteil unserer Listen ist, dass wir sie jetzt mithilfe der `pandas` Bibliothek in einen Dataframe, also eine tabellarische Struktur, überführen können. Das machen wir mit `pd.DataFrame`, hier können wir z.B. auch Namen für die Spalten (`columns`) mit angeben. Mit der `head()`-Funktion erhalten wir eine Übersicht über die ersten Einträge unseres Dataframes.

In [10]:
#import lists as cols to a pandas dataframe
xml_text_dataframe = pd.DataFrame(list(zip(xml_nodes, xml_tags, xml_text)), columns =["nodes", "tags", "text"])
xml_text_dataframe.head()

,nodes,tags,text
0,[],{http://www.tei-c.org/ns/1.0}title,H. P. Lovecraft: Astronomical Observations (NAME)
1,[],{http://www.tei-c.org/ns/1.0}expan,Observed
2,[],{http://www.tei-c.org/ns/1.0}abbr,Obs’d
3,[],{http://www.tei-c.org/ns/1.0}expan,Observed
4,[],{http://www.tei-c.org/ns/1.0}abbr,Obs’d


Seltsam - sind unsere `nodes` hier etwa verschwunden und nur noch leere Listen `[]` enthalten? Werfen wir nochmal einen genaueren Blick darauf und lassen uns den ersten Eintrag aus der Tabelle ausgeben:

In [11]:
print(xml_text_dataframe["nodes"][0])

<Element '{http://www.tei-c.org/ns/1.0}title' at 0x7f8a382fd950>


Puh, alles noch da! 😅 When in doubt, use print: https://www.reddit.com/r/ProgrammerHumor/comments/ntro76/all_the_print_statements/

## Teil 2: Named Entity Recognition (NER) für XML 

### 5. NER auf einem Text anwenden
Um Named Entity Recognition (NER), also ein Verfahren zur Erkennung benannter Entitäten wie z.B. Personen-, Orts- oder Organisationsnamen, auf den textuellen Daten anzuwenden, müssen wir zunächst ein entsprechendes Modell laden. Wir haben bereits weiter oben das vortrainerte `en_core_web_sm` Modell von [SpaCy](https://spacy.io) installiert, jetzt laden wir dieses mit `spacy.load()`. Als Textbeispiel (`xml_text_example`) nehmen wir den Titel aus der ersten Zeile des DataFrames und wenden das Modell darauf an. Nun können wir uns beispielsweise für alle im Titel erkannten Entitäten deren Inhalt (`text`), Position (`start_char` = erstes Zeichen im Eingabetext, `end_char` = letztes Zeichen im Eingabetext) sowie das vergebene `label_` (z.B. `PER` für Person, `ORG` für Organisation) ausgeben lassen.

In [12]:
nlp = spacy.load("en_core_web_sm")

In [13]:
xml_text_example = xml_text_dataframe["text"][0]
print(xml_text_example)
doc = nlp(xml_text_example)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

H. P. Lovecraft: Astronomical Observations (NAME)
H. P. Lovecraft 0 15 PERSON
NAME 44 48 ORG


Es wurden zwei Entitäten im entsprechenden Text gefunden! Bei NAME handelt es sich wohl um einen Fehler, aber `H. P. Lovecraft` als Person auszuzeichnen erscheint sinnvoll. Um den zugehörigen Knoten im eingelesenen XML zu aktualisieren, definieren wir zwei weitere Variablen: `xml_title_node` und `per_to_be_added`. `xml_title_node` ist der Titelknoten aus der 1. Zeile des DataFrames und `per_to_be_added` die korrekt erkannte Person, die wir nun neu einbinden wollen.

In [14]:
xml_title_node = xml_text_dataframe["nodes"][0]
per_to_be_added = doc.ents[0]

In [15]:
print(per_to_be_added.end_char)

15


Jetzt aktualisieren wir den Knoten und fügen ein NER-Tag hinzu. Dazu lesen wir zunächst den ursprünglichen Text bis zur Startposition der erkannten Entität aus. Dann nutzen wir `SubElement`, um ein Unterelement, in dem Fall `persName` hinzuzufügen. Anschließend erhält `persName` den Text der Entität und wir fügen nachfolgend den restlichen Text als `tail` hinzu.

In [16]:
xml_title_node.text = xml_text_example[0:per_to_be_added.start_char] #set text within node equal to all chars until PER entity starts
entity_tag = ET.SubElement(xml_title_node, 'persName') #add a nested tag for new entity
entity_tag.text = per_to_be_added.text #set entity tag text equal to ent.text for tag
entity_tag.tail = xml_text_example[per_to_be_added.end_char:len(xml_text_example)] #now, add the rest of the text        

Zum Schluss checken wir nochmal, ob persName innerhalb des `title`-Knotens entsprechend hinzugefügt wurde, indem wir ihn mit `root.findall()` auslesen und durch dessen Kindelemente iterieren: 

In [17]:
#check if node changed
for i in root.findall(".//title", namespaces)[0]:
    print(i)

<Element 'persName' at 0x7f8a382341d0>


Da wir im nächsten Schritt NER auf den gesamten Beispieltext anwenden wollen, bereinigen wir das Dokument vorerst wieder vom `persName`-Element (es diente hier nur der Demonstration). 

In [18]:
xml_title_node.text = xml_text_example
title_children = root.findall(".//title", namespaces)[0]
for i in title_children:
    title_children.remove(i)
print(xml_title_node.text)

H. P. Lovecraft: Astronomical Observations (NAME)


### 6. NER auf viele Textelemente gleichzeitig anzuwenden

Wir können auch eigene Funktionen definieren, um unseren Code u.a. leichter wiederverwenden zu können und besser zu strukturieren. Hier ein einfaches Beispiel, wie so eine Funktion aussehen könnte. Für den Funktionsparameter `xml_table` können wir nun beispielsweise unseren `xml_text_dataframe` einsetzen und uns die ersten 5 Texteinträge aus der Tabelle ausgeben lassen:

In [19]:
def print_text(xml_table):
    num = 1
    for text in xml_table["text"]:
        print("Text Snippet #" + str(num) + ": " + text)
        num += 1

print_text(xml_text_dataframe[:5])

Text Snippet #1: H. P. Lovecraft: Astronomical Observations (NAME)
Text Snippet #2: Observed
Text Snippet #3: Obs’d
Text Snippet #4: Observed
Text Snippet #5: Obs’d


In ähnlicher Weise können wir nun auch unseren bisherigen NER Code in einer Funktion unterbringen. Mit `enumerate()` werden die einzelnen `text` innerhalb des for-Loops durchgezählt und wir können die zugehörige Position (`index`) ansprechen - diese Position benötigten wir, um die entsprechenden NER-Tags an der jeweils richtigen Stelle im XML-Dokument einfügen zu können.

In [20]:
def ner_text(xml_table):
    for index, text in enumerate(xml_table["text"]):
        doc = nlp(text)
        for ent_count, ent in enumerate(doc.ents):
            current_xml_node = xml_table["nodes"][index]
            
            # if else statements for different positions within text
            if ent_count==0: #first additional tag
                current_xml_node.text = text[0:ent.start_char]   
            else: #last tag
                current_xml_node.text = current_xml_node[last_end_char:ent.start_char]
            entity_tag = ET.SubElement(current_xml_node,  'name') 
            entity_tag.set('n', 'ner') #add additional attributes for more information
            entity_tag.set('type', str(ent.label_))
            entity_tag.text = ent.text 
            
            if len(doc.ents) > ent_count+1: #if there are more tags to add, only read until start_char of next ent
                entity_tag.tail = text[ent.end_char:doc.ents[ent_count+1].start_char]
            else:
                entity_tag.tail = text[ent.end_char:len(text)]
                
            last_end_char = ent.end_char

ner_text(xml_text_dataframe)

### 7. Vergleich: NER auf plaintext anwenden

In [21]:
import re

with open("data/input.txt", encoding="utf-8") as file:
    textfile = file.read()

textfile = textfile.strip()
textfile = re.sub(r'\s+', ' ', textfile)

print(textfile)

Astronomical Observations Made By H.P. Lovecraft, 598, Angell St., Providence, R.I. U.S.A. Years: 1909 1910 1911 1912 1913 1914 1915 Comet, Halleys, Observed May 26, 1910 p. 1 Comet, Delavan’s, Observed September 16, 1914 p. 7 Eclipse of ☽︎ - June 3, 1909 p. a. Eclipse of ☽︎ - March 11-12, 1914 p. 6 Halley's Comet - May 26, 1910 p. 1 Halo around ☽︎, February 1, 1912 p 3. Mars, Occultation of, Observed September 1, 1909 p. 1 Moon, Eclipse of - June 3, 1909 p. a. Moon, Halo around, February 1, 1912 p. 3 Mercury near Moon - February 26, 1914 pp. 5-6 Occultation of Mars - September 1, 1909 p. 1 Paraselenae - Lunar Halo - February 1, 1912 p. 3 Special Observation June 3 - 1909 Moon’s Eclipse Clouds interfered, but several glimpses were obtained - Total 7.58 - ASTRONOMICAL OBSERVATIONS 1909 Begun September 1, 1909 September 1 - Observed Occultation of Mars by Moon’s bright limb at 8h 58m occultation took 40s. Ending at 9:57 was not observed. Shy hazy. Moon fair. Observed Mare Humorum Cleomed

In [22]:
doc_textfile = nlp(textfile)

for ent in doc_textfile.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

H.P. Lovecraft 34 48 ORG
598 50 53 CARDINAL
Angell St. 55 65 ORG
Providence 67 77 GPE
R.I. 79 83 GPE
1909 1910 1911 1912 1913 1914 1915 98 132 DATE
Halleys 140 147 PERSON
May 26, 1910 158 170 DATE
1 174 175 CARDINAL
Delavan’s 183 192 ORG
September 16, 1914 203 221 DATE
7 225 226 CARDINAL
March 11-12, 1914 278 295 DATE
6 299 300 CARDINAL
Halley 301 307 ORG
1910 326 330 DATE
1 334 335 CARDINAL
February 1 352 362 DATE
1912 364 368 DATE
3 371 372 CARDINAL
September 1, 1909 405 422 DATE
1 426 427 CARDINAL
1909 455 459 DATE
Halo 472 476 PERSON
February 1, 1912 485 501 DATE
3 505 506 CARDINAL
Mercury 507 514 ORG
Moon 520 524 GPE
1914 540 544 DATE
5 549 550 CARDINAL
1909 588 592 DATE
1 596 597 CARDINAL
February 1 625 635 DATE
1912 637 641 DATE
3 645 646 CARDINAL
June 3 - 1909 Moon’s Eclipse Clouds 667 702 DATE
7.58 758 762 CARDINAL
1909 791 795 DATE
Begun September 1, 1909 September 1 - Observed Occultation of Mars 796 862 EVENT
Moon 866 870 PERSON
8h 58 888 893 CARDINAL
40s 912 915 DATE
9:57 

### 8. Ergebnisse exportieren
Am Ende kann der aktualisierte XML-Baum mit `tree.write()` wieder in eine XML-Datei geschrieben werden. Damit der Namensraum-Präfix nicht in den Elementen auftaucht, können wir ihn mit `register_namespace` vorher entsprechend angeben.

In [23]:
#write result to output file
ET.register_namespace("", "http://www.tei-c.org/ns/1.0")
tree.write('data/output.xml', xml_declaration=True, encoding='utf-8', method='xml')
#TODO: add PI/statement from L2: <?xml-model href="hpl_ao.rng" type="application/xml" schematypens="http://relaxng.org/ns/structure/1.0"?>

## Teil 3: Weitere Natural Language Processing (NLP) Anwendungsfälle und Analysen 

In den vorherigeren Teilen haben wir gezeigt, (1) wie wir XML einlesen und verarbeiten können, und (2) wie wir ein bereits vortrainertes NER-Modell auf einzelne Bestandteile des XML-Dokuments oder einen vorab aus dem XML extrahierten Plaintext anwenden können. Natürlich wären basierend auf den vorangegangenen Verarbeitungsschritten noch zahlreiche andere Analysen möglich, mit denen wir das XML potenziell anreichern könnten. Hier wollen wir ausblickend anhand einiger Beispiele aufzeigen, welche weiterführenden Analyse noch möglich wären. Im Folgenden dazu einige häufige Szenarien:

- **Sentiment-Analyse**: Messung von Stimmung (Emotion)
- **Kollokationen**: gehäuftes Auftreten von Wortpaaren
- **KWIC**: Kontexte von Begriffen im Text
- **POS-Tagging**: Auswertung von Wortarten, Normalisierung

### Vorbereitung des Plaintexts für weitere Analysen

Schauen wir uns erst einmal an, wie wir den Text ggf. noch für weitere Analyse aufbereiten können. Wir nutzen unten einen regelbasierten Ansatz (mithilfe von re), um Satzstrukturen zu erkennen und bauen dann daraus wieder einen pandas DataFrame. Alternativ hätten wir auch den sogenannten [Sentencizer](https://spacy.io/api/sentencizer) von spaCy hierfür nutzen können. Es gibt also meistens nicht den einen Ansatz für ein Problem, und ggf. muss man auch Verschiedenes einfach ausprobieren. Schauen wir uns die Sätze einmal und gehen dann zum nächsten Schritt über.

In [24]:
# Alles aus textfile → zu DataFrame → komplett anzeigen (scrollbar)

# Sätze splitten und in DataFrame-Variable umwandeln
sentences = [s.strip() for s in re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', textfile) if s.strip()]
plaintext_df = pd.DataFrame({'nodes': range(len(sentences)), 'text': sentences})

# Alle rows anzeigen
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):
    display(plaintext_df)

print(f" {len(plaintext_df)} Sätze komplett angezeigt")

,nodes,text
0,0,"Astronomical Observations Made By H.P. Lovecraft, 598, Angell St., Providence, R.I. U.S.A. Years: 1909 1910 1911 1912 1913 1914 1915 Comet, Halleys, Observed May 26, 1910 p."
1,1,"1 Comet, Delavan’s, Observed September 16, 1914 p."
2,2,"7 Eclipse of ☽︎ - June 3, 1909 p."
3,3,a.
4,4,"Eclipse of ☽︎ - March 11-12, 1914 p."
5,5,"6 Halley's Comet - May 26, 1910 p."
6,6,"1 Halo around ☽︎, February 1, 1912 p 3."
7,7,"Mars, Occultation of, Observed September 1, 1909 p."
8,8,"1 Moon, Eclipse of - June 3, 1909 p."
9,9,a.


 90 Sätze komplett angezeigt


### 9. Sentiment-Analyse mit VADER (NLTK)

Bei der Sentimentanalyse geht es um eine automatische Stimmungserkennung, die stark verwandt mit der sogenannten Emotionsanalyse ist. Beim Sentiment geht allerdings lediglich um die Einteilung in positive (Werte zwischen `0` und `+1`), neutrale (`0`) und negative (Werte zwischen `-1` und `0`) Stimmungslagen. 

Für die Sentimentanalyse verwenden wir das Natural Language Toolkit, kurz NLTK. Neben dem Installieren (bereits erfolgt) und Importieren der nltk Bibliothek müssen wir noch einmalig das VADER Lexikon herunterladen.

In [25]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [26]:
#nltk.download('vader_lexicon', quiet=True)

In [27]:
# VADER initialisieren
sia = SentimentIntensityAnalyzer()

# Sentiment auf alle Textelemente berechnen
sentiment_scores = []
for idx, row in plaintext_df.iterrows():
    text = row['text']
    scores = sia.polarity_scores(text)
    sentiment_scores.append({
        'node': row['nodes'],
        'text': text,
        'sentiment_compound': scores['compound'],
        'sentiment_label': 'positiv' if scores['compound'] > 0.05 else \
                           'negativ' if scores['compound'] < -0.05 else 'neutral'
    })

sentiment_df = pd.DataFrame(sentiment_scores)
sentiment_df.head()

# CSV-Export: Sentiment-Scores
sentiment_df.to_csv('data/nlp_output/sentiment_plaintext.csv', index=False, encoding='utf-8')
print(sentiment_df['sentiment_label'].value_counts())

sentiment_label
neutral    58
positiv    26
negativ     6
Name: count, dtype: int64


### 10. Kollokationen & KWIC (NLTK)

Bei der Analyse der **Kollokationen** untersuchen wir, welche Wortpaare im Text besonders häufig zusammen auftreten. Mit **KWIC (Keyword in Context)** können wir uns auf einen Blick anzeigen lassen, in welchen unterschiedlichen Kontexten ein Begriff innerhalb eines Textes auftaucht.

In [28]:
# Kollokationen über alle Textelemente
from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures
from nltk.tokenize import word_tokenize

nltk.download('punkt_tab', quiet=True)

# Alle Tokens sammeln
all_tokens = []
for text in plaintext_df['text']:
    tokens = word_tokenize(text.lower())
    tokens_clean = [t for t in tokens if t.isalpha()]
    all_tokens.extend(tokens_clean)

# Alle Kollokationen berechnen
bigram_finder = BigramCollocationFinder.from_words(all_tokens)
alle_kollokationen_scored = bigram_finder.score_ngrams(BigramAssocMeasures.likelihood_ratio)  # ALLE mit Scores!

# Top 10 anzeigen
top10_kollokationen = bigram_finder.nbest(BigramAssocMeasures.likelihood_ratio, 10)
print("Top 10 Kollokationen (Lovecraft-Stil):")
for kollokation in top10_kollokationen:
    print(f"  {kollokation}")

# KWIC für Beispiel-Keyword "comet" (muss dann dementsprechend ersetzt/ angepasst werden, falls anderes Keyword gewünscht)
def kwic(text_list, keyword, context_size=5):
    keyword = keyword.lower()
    results = []
    for text in text_list:
        tokens = word_tokenize(text.lower())
        for i, token in enumerate(tokens):
            if token == keyword:
                start = max(0, i-context_size)
                end = min(len(tokens), i+context_size+1)
                context = ' '.join(tokens[start:end])
                results.append(context)
                print(f"...{context}...")
    return results

print("\nKWIC für 'comet':")
kwic_results = kwic(plaintext_df['text'].tolist(), 'comet')

# CSV-Export: alle Kollokationen mit Scores
kollokationen_df = pd.DataFrame(alle_kollokationen_scored, columns=['kollokation', 'score'])
kollokationen_df['kollokation'] = kollokationen_df['kollokation'].astype(str)
kollokationen_df.to_csv('data/nlp_output/kollokationen_plaintext.csv', index=False)

# CSV-Export: KWIC für "comet"
kwic_df = pd.DataFrame({'kwic_comet': kwic_results})
kwic_df.to_csv('data/nlp_output/kwic_comet_plaintext.csv', index=False)

Top 10 Kollokationen (Lovecraft-Stil):
  ('prism', 'binoculars')
  ('delavan', 's')
  ('attracted', 'much')
  ('mare', 'humorum')
  ('much', 'attention')
  ('local', 'time')
  ('opera', 'glasses')
  ('to', 'keep')
  ('s', 'comet')
  ('eclipse', 'of')

KWIC für 'comet':
...1911 1912 1913 1914 1915 comet , halleys , observed may...
...1 comet , delavan ’ s ,...
...6 halley 's comet - may 26 , 1910...
...26 observed halley ’ s comet 8.45-9.30 p.m. visible in west...
...looking westward head of comet ι θ ζ ρ position...
...- observed delavan ’ s comet & heavens in general ....
...delavan ’ s comet lay in r.a. 10h 08m...
...plough comet northeast horizon it had about...
...of 50 & 100 the comet was indeed a beautiful sight...
...- observed delavan ’ s comet 4-5 a.m. also moon ....
...comet appears brighter as moon wanes...


### 11. POS-Tagging & Feature-Extraktion (spaCy)

Part-of-Speech oder POS-Tagging bezeichnet das Annotieren von Texten mit Wortarten. Mit unserem spaCy erhalten wir nach dem Prozessieren unseres Textes neben den NER-Tags auch direkt die POS-Tags. Wir können so den Text nicht nur weiter aufbereiten, sondern auch z.B. bestimmte Wortarten, die eigene Funktionen im Text erfüllen, extrahieren: 
* Adjektive (Bewertung/Sentiment)
* Verben (Aktionen)
* Substantive (Fokus)

Mit `token.lemma_.lower()` können wir die Tokens für darauf aufbauende Analysen normalisieren bzw. zusammenführen. So werden beispielsweise `commet` und `comet` in der abschließenden Auswertung als gleiches Token gezählt. POS-Tags erscheinen für manche Anwendungsfälle zunächst vielleicht nicht relevant, können aber für andere NLP-Verfahren hilfreiche Informationen über die Sprache des Textes liefern. 

In [29]:
# POS-Tagging
pos_features = []
for idx, row in plaintext_df.iterrows():
    text = row['text']
    doc = nlp(text)
    
# Bereinigte Features extrahieren (keine URLs, Zahlen, etc.)
    adjectives = [
        token.lemma_.lower() 
        for token in doc 
        if token.pos_ == 'ADJ' 
        and token.is_alpha 
        and not token.like_url 
        and len(token.text) > 2
    ]
    verbs = [
        token.lemma_.lower() 
        for token in doc 
        if token.pos_ == 'VERB' 
        and token.is_alpha 
        and len(token.text) > 2
    ]
    nouns = [
        token.lemma_.lower() 
        for token in doc 
        if token.pos_ in ['NOUN', 'PROPN'] 
        and token.is_alpha 
        and len(token.text) > 2
    ]
    
    pos_features.append({
        'node': row['nodes'],
        'text': text,
        'adjectives': adjectives,
        'verbs': verbs,
        'nouns': nouns,
        'num_adjectives': len(adjectives),
        'num_verbs': len(verbs),
        'num_nouns': len(nouns)
    })

pos_df = pd.DataFrame(pos_features)
display(pos_df.head())  # Besser als print

# Top Listen 
print("\nTop 10 Adjektive (Lovecrafts Bewertungen):")
all_adjs = [adj for row in pos_df['adjectives'] for adj in row]
top_adjs = pd.Series(all_adjs).value_counts()
print(top_adjs.head(10))

print("\nTop 10 Verben:")
all_verbs = [v for row in pos_df['verbs'] for v in row]
top_verbs = pd.Series(all_verbs).value_counts()
print(top_verbs.head(10))

print("\nTop 10 Substantive:")
all_nouns = [n for row in pos_df['nouns'] for n in row]
top_nouns = pd.Series(all_nouns).value_counts()
print(top_nouns.head(10))

# CSV-Export: POS-Ergebnisse
pos_df.to_csv('data/nlp_output/pos_tagging_plaintext.csv', index=False, encoding='utf-8')


,node,text,adjectives,verbs,nouns,num_adjectives,num_verbs,num_nouns
0,0,Astronomical Observations Made By H.P. Lovecra...,[],[make],"[astronomical, observations, lovecraft, angell...",0,1,10
1,1,"1 Comet, Delavan’s, Observed September 16, 191...",[observed],[],"[comet, delavan, september]",1,0,3
2,2,"7 Eclipse of ☽︎ - June 3, 1909 p.",[],[],"[eclipse, june]",0,0,2
3,3,a.,[],[],[],0,0,0
4,4,"Eclipse of ☽︎ - March 11-12, 1914 p.",[],[],"[eclipse, march]",0,0,2



Top 10 Adjektive (Lovecrafts Bewertungen):
bright          7
visible         5
faint           3
observed        3
terrestrial     3
beautiful       3
astronomical    3
much            2
broad           2
prominent       2
Name: count, dtype: int64

Top 10 Verben:
observe       8
appear        5
see           5
earthshine    3
define        3
keep          3
look          2
make          2
continue      2
add           2
Name: count, dtype: int64

Top 10 Substantive:
moon           24
comet          11
observed        8
february        8
time            8
september       7
halo            7
star            6
observation     6
position        5
Name: count, dtype: int64
